In [1]:
# modules we'll use
import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

# read in all our data
HPP_data = pd.read_csv("../train.csv")

# set seed for reproducibility
np.random.seed(0)

In [2]:
#Impute values which are missing
HPP_data['LotFrontage'] = HPP_data['LotFrontage'].fillna('0')
HPP_data['Alley'] = HPP_data['Alley'].fillna('NoAlley')
HPP_data['BsmtQual'] = HPP_data['BsmtQual'].fillna('NoBasement')
HPP_data['BsmtCond'] = HPP_data['BsmtCond'].fillna('NoBasement')
HPP_data['BsmtExposure'] = HPP_data['BsmtExposure'].fillna('NoBasement')
HPP_data['BsmtFinType1'] = HPP_data['BsmtFinType1'].fillna('NoBasement')
HPP_data['BsmtFinType2'] = HPP_data['BsmtFinType2'].fillna('NoBasement')
HPP_data['FireplaceQu'] = HPP_data['FireplaceQu'].fillna('NoFireplace')
HPP_data['GarageType'] = HPP_data['GarageType'].fillna('NoGarage')
HPP_data['GarageYrBlt'] = HPP_data['GarageYrBlt'].fillna(-1)
HPP_data['GarageFinish'] = HPP_data['GarageFinish'].fillna('NoGarage')
HPP_data['GarageQual'] = HPP_data['GarageQual'].fillna('NoGarage')
HPP_data['GarageCond'] = HPP_data['GarageCond'].fillna('NoGarage')
HPP_data['PoolQC'] = HPP_data['PoolQC'].fillna('NoPool')
HPP_data['Fence'] = HPP_data['Fence'].fillna('NoFence')
HPP_data['MiscFeature'] = HPP_data['MiscFeature'].fillna('NoMiscFeature')

HPP_data['MasVnrArea'] = HPP_data['MasVnrArea'].fillna(HPP_data['MasVnrArea'].mean())
HPP_data['MasVnrType'] = HPP_data['MasVnrType'].fillna(HPP_data['MasVnrType'].mode()[random.randint(0, len(pd.Series(HPP_data['MasVnrType']).mode()) - 1)])
HPP_data['Electrical'] = HPP_data['Electrical'].fillna(HPP_data['Electrical'].mode()[random.randint(0, len(pd.Series(HPP_data['Electrical']).mode()) - 1)])



In [3]:
# Identify categorical columns
categorical_columns = HPP_data.select_dtypes(include=['object']).columns

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Iterate through the categorical columns
for column in categorical_columns:
    # Convert all values to strings and then fit and transform the LabelEncoder
    HPP_data[column] = label_encoder.fit_transform(HPP_data[column].astype(str))

In [4]:
#baseline score for features
X = HPP_data.copy()
y = X.pop("SalePrice")

# Train and score baseline model
baseline = RandomForestRegressor(criterion="absolute_error", random_state=0)
baseline_score = cross_val_score(
    baseline, X, y, cv=5, scoring="neg_mean_absolute_error"
)
baseline_score = -1 * baseline_score.mean()

print(f"MAE Baseline Score: {baseline_score:.4}")

MAE Baseline Score: 1.776e+04


In [5]:
X = HPP_data.copy()
y = X.pop("SalePrice")

# Create synthetic features
X["TimeOld"] = X['YrSold'] + X["MoSold"]/12
#TODO: compute the total square feet in the house
X['TotalSF'] = X['TotalBsmtSF']+ ['GarageArea']
#TODO: compute total square feet in total property

#TODO: compute total number of rooms in house

# Train and score model on dataset with additional ratio features
model = RandomForestRegressor(criterion="absolute_error", random_state=0)
score = cross_val_score(
    model, X, y, cv=5, scoring="neg_mean_absolute_error"
)
score = -1 * score.mean()

print(f"MAE Score with Ratio Features: {score:.4}")

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('int64'), dtype('<U10')) -> None